# Convert Binary to Fits 

<br>

[Create Fits Files](#Create-Fits-Files)

[Find Molecular Dominated Sims](#Find-Molecular-Dominated-Sims)

<br>

This code first loads in all the files (density, dust temperature, gas temperature, and the abundances of CO, H2, and HP. It is worth noting that the density, and abunances are all converted to abundances relative to total hydrogen nuclei. 

If you want to make H1 cubes, use "Making HI Files Feedback Dominated"

If you want to find the sims that have a lot of molecular H2, go to "Find Molecular Dominated Sims"

In [1]:
!ls

Feedback Dominated Binary to Fits.ipynb
Making HI Files - Feedback Dominated.ipynb
density_grid_x1470y1400z1400
tdust_grid_x1470y1400z1400
temp_grid_x1470y1400z1400
xCO_grid_x1470y1400z1400
xH2_grid_x1470y1400z1400


In [2]:
################################################
# 
# Imports
#
################################################

import numpy as np 
from astropy.io import fits 
import glob

################################################
#
# Constants 
#
################################################

#Units to convert AREPO units to physical units 

ulength=3.0856e+20 #internal unit length in centimeters
umass=1.9910000e+33 #internal unit mass in grams
uvel=1.0e+5 #internal unit velocity in cm/s
udensity=umass/ulength**3 #internal unit density in g/cm^-3
utime=3.0856000e+15 #internal unit time in seconds 
mp=1.67e-24 #mass of proton
dusttogas= 0.01 #dust to gas mass ratio
n = 500 #size of grids is 500 x 500 x 500 voxels, where each voxel = 1 pc in length


################################################
# 
# Load in File Lists 
#
################################################

# Making file lists for each file type
fn_density = glob.glob("density_grid*")  # tot density of gas (in sim units)
fn_tdust   = glob.glob("tdust_grid*")    # temp of dust (in K)
fn_temp    = glob.glob("temp_grid*")     # temp of gas (in K)
fn_xH2     = glob.glob("xH2_grid*")      # abundance of H2 wrt total vol density of hydrogen nuclei (nH2 = nhtot * xH2)
fn_xHP     = glob.glob("xHP_grid*")      # abundance of H+ wrt total vol density of hydrogen nuclei (nHP = nhtot * xHP)
fn_xCO     = glob.glob("xCO_grid*")      # abundance of CO wrt total vol density of hydrogen nuclei (nCO = nhtot * xCO)
#fn_vel     = glob.glob("velocity_grid*") # 3d space velocities in sim units (vx,vy,vz)



# Create Fits Files

In [3]:
################################################
# 
# Convert files to fits 
#
################################################

# Loop through each file list
## Open file, read into an array, reshape the array
## If necessary, convert from sim units to real units 
## If necessary, convert to other units (such as Hydrogen nuclei)
## Close file and write into a fits file

##############################
## Density
##############################
for file_name in fn_density: 
    
    fdensity = open(file_name, "r") # r is the default 
    
    # Right now the file is still binary, but we want it to be
    # an np array 
    
    # read in binary file to an array
    d = np.fromfile(fdensity, dtype=np.float32)
    d = np.delete(d, [0,1,2]) #delete first three entrees (which are not data)
    
    # reshape the 1D array to 3d (n x n x n)
    d = d.reshape([n,n,n]) # n = 500
    
    # convert from AREPO units to actual density (g/cm^3)
    rhogas = d*udensity
    
    # convert from total gas density to nubmber density of hydrogen nuclei 
    # the factor is 1.4 (mean molecular weight of H)
    ## mean molecular weight = average mass in terms of hydrogen masses
    ## mean molecular weight^-1 = 1/(1+4x_he), x_he = 0.1m 
    nhtot = rhogas/(1.4 * mp)
    
    fdensity.close() #closes the file
    
    # make a fake hearder so its easy to view (?)
    hdu = fits.PrimaryHDU(nhtot)
    
    # write to a fits file 
    file_name = file_name.replace('density_grid_','nhtot_')+'.fits'
    hdu.writeto(file_name,overwrite=True)
    
#print("Finished Sucessfully")

In [4]:
##############################
## Temperature Files
##############################

##############################
## Dust Temperature 
##############################
for file_name in fn_tdust: 
    
    f_tdust = open(file_name,'r')
    
    d = np.fromfile(f_tdust, dtype=np.float32)
    d = np.delete(d,[0,1,2])
    d = d.reshape([n,n,n])
    
    f_tdust.close()
    
    hdu = fits.PrimaryHDU(d)
    file_name = file_name.replace('_grid_','_fits_')+'.fits'
    hdu.writeto(file_name,overwrite=True)
    
##############################
## Gas Temperature 
##############################
for file_name in fn_temp: 
    
    f_temp = open(file_name,'r')
    
    d = np.fromfile(f_temp, dtype=np.float32)
    d = np.delete(d,[0,1,2])
    d = d.reshape([n,n,n])
    
    f_temp.close()
    
    hdu = fits.PrimaryHDU(d)
    file_name = file_name.replace('_grid_','_fits_')+'.fits'
    hdu.writeto(file_name,overwrite=True)

In [5]:
##############################
## Abundance Files
##############################

# Abundance graphs must be converted from xH2, xHP, xCO to nH2, nHP, nCO
# We do this by multiplying the corrsponding nhtot fits by d in each

fn_nhtot = glob.glob("nhtot*")

for fits_files in fn_nhtot:
    
    last_15 = fits_files[-20:-5] # See below cell for justification
    print(last_15)
    nhtot = fits.getdata(fits_files) # To multiply the d's by 
    
    ##############################
    ## H2 Abundance
    ##############################
    for file_name in fn_xH2: 
    
        if(file_name[-15:] == last_15):
            
            f_H2 = open(file_name,'r')
    
            d = np.fromfile(f_H2, dtype=np.float32)
            d = np.delete(d,[0,1,2])
            d = d.reshape([n,n,n])
            
            nH2 = d*nhtot
    
            f_H2.close()
    
            hdu = fits.PrimaryHDU(nH2)
            file_name = file_name.replace('xH2_grid_','nH2_')+'.fits'
            hdu.writeto(file_name,overwrite=True)
        
    ##############################
    ## H+ Abundance
    ##############################
    for file_name in fn_xHP: 
    
        if(file_name[-15:] == last_15):
            
            f_HP = open(file_name,'r')
    
            d = np.fromfile(f_HP, dtype=np.float32)
            d = np.delete(d,[0,1,2])
            d = d.reshape([n,n,n])
        
            nHP = d*nhtot
            
            f_HP.close()
    
            hdu = fits.PrimaryHDU(nHP)
            file_name = file_name.replace('xHP_grid_','nHP_')+'.fits'
            hdu.writeto(file_name,overwrite=True)
    
    ##############################
    ## CO Abundance
    ##############################
    for file_name in fn_xCO: 
    
        if(file_name[-15:] == last_15):
            
            f_CO = open(file_name,'r')
    
            d = np.fromfile(f_CO, dtype=np.float32)
            d = np.delete(d,[0,1,2])
            d = d.reshape([n,n,n])
            
            nCO = d*nhtot
            
            f_CO.close()
    
            hdu = fits.PrimaryHDU(nCO)
            file_name = file_name.replace('xCO_grid_','nCO_')+'.fits'
            hdu.writeto(file_name,overwrite=True)

x1470y1400z1400
